In [ ]:
# == 모델 실행 == 
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

# 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("./food_chatbot_model")
model = GPT2LMHeadModel.from_pretrained("./food_chatbot_model")
model.to(device)
model.eval()

def generate_response(input_text):
    input_ids = tokenizer.encode(f"질문: {input_text}\n답변: ", return_tensors='pt').to(device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.92,
            temperature=0.6,
            repetition_penalty=1.2,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )
    output = tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    # 답변 부분만 추출
    response = output.split("답변:")[1].strip()
    return response


print("음식 추천 챗봇입니다. 종료하려면 '종료'를 입력하세요.")

while True:
    user_input = input("사용자: ")
    if user_input.strip().lower() == "종료":
        print("챗봇: 대화를 종료합니다. 좋은 하루 되세요!")
        break
    response = generate_response(user_input)
    print(f"챗봇: {response}")
